In [15]:
# Imports

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

In [30]:
# Traffic Simulation

class LoadGenerator:
    """
    Generate load patterns with added Gaussian noise.
    """
    def __init__(self, mean=0, std=1, seed=None, freq=1):
        self.time = 0
        self.freq = freq
        self.mean = mean
        self.std = std
        self.rng = np.random.default_rng(seed)
        
    def gen_load_const(self, steps):
        values = self.rng.normal(self.mean, self.std, size=(steps, 1))
        
        times = self.time + np.cumsum(self.rng.exponential(scale=1/self.freq, size=(steps, 1)), axis=0)
        self.time = times[-1]
        
        category = np.zeros((steps, 1))
        
        return np.concatenate((times, values, category), axis=1)
    
    
    def gen_load_change_freq(self, steps, alpha):
        values = self.rng.normal(self.mean, self.std, size=(steps, 1))
        
        new_freq = (1 + alpha)*self.freq
        freqs = np.linspace(self.freq, new_freq, steps + 2)[1:-1]
        times = self.time + np.cumsum([self.rng.exponential(scale=1/freq) for freq in freqs]).reshape((steps, 1))
        self.time = times[-1]
        self.freq = freqs[-1]
                
        category = np.ones((steps, 1))
                
        return np.concatenate((times, values, category), axis=1)
    
    
    def gen_load_change_val(self, steps, delta):
        new_mean = self.mean + delta
        mean = np.linspace(self.mean, new_mean, steps + 2)[1:-1]
        noise = self.rng.normal(0, self.std, steps)
        values = (mean + noise).reshape((steps, 1))
        self.mean = new_mean
        
        times = self.time + np.cumsum(self.rng.exponential(scale=1/self.freq, size=(steps, 1)), axis=0)
        self.time = times[-1]
        
        category = 2*np.ones((steps, 1))
        
        return np.concatenate((times, values, category), axis=1)
    
    
def generate_stream(length=10000, episode_length=500, episode_variability=0.5, intensity=0.1,
                    start_mean=0, noise_std=1, abs_delta_mean=None, delta_var=None,
                    abs_alpha_min=0.2, abs_alpha_max=0.9, seed=None):
    """
    Generate a data stream with load changes and peaks.
    """
    rng = np.random.default_rng(seed)
    lgen = LoadGenerator(start_mean, noise_std, rng)
    
    min_ep_len = np.floor(episode_length*(1 - episode_variability)).astype(int)
    max_ep_len = np.ceil(episode_length*(1 + episode_variability)).astype(int)
    if not abs_delta_mean:
        abs_delta_mean = 2*noise_std
    if not delta_var:
        delta_var = noise_std
    
    stream = np.empty((0, 3))
    while len(stream) < length:
        ep_len = rng.integers(min_ep_len, max_ep_len)
        ep_category = rng.choice(a=3, p=[1 - intensity] + [intensity/2 for _ in range(2)])
        if ep_category == 0:
            episode = lgen.gen_load_const(ep_len)
            stream = np.concatenate((stream, episode))
        elif ep_category == 1:
            delta = rng.choice([-1, 1])*rng.normal(abs_delta_mean, delta_var)
            episode = lgen.gen_load_change_val(ep_len, delta)
            stream = np.concatenate((stream, episode))
        elif ep_category == 2:
            alpha = rng.choice([-1, 1])*rng.uniform(abs_alpha_min, abs_alpha_max)
            episode = lgen.gen_load_change_freq(ep_len, alpha)
            stream = np.concatenate((stream, episode))
        else:
            raise Exception("Unexpected episode category")
        
    return stream[:, :2], stream[:, 2]

In [29]:
# Plotting

def plot_stream(stream, signal=None, labels=None, colors=None,
                figsize=(11, 5), markersize=1, linewidth=0.1, ax=None):
    if not ax:
        fig, ax = plt.subplots(figsize=figsize, dpi=200)
    if signal is not None:
        if not (labels and colors):
            raise Exception("Need labels and colors for provided signal")
        elif not len(labels) == len(colors):
            raise Exception("Must provide same number of colors as labels")
            
        colormap = dict(enumerate(colors))
        categorymap = dict(enumerate(labels))
        patches = [mpatches.Patch(color=c, label=l) for c, l in zip(colormap.values(), categorymap.values())]
        colorlist = list(map(lambda x: colormap[x], signal))
        ax.scatter(stream[:,0], stream[:,1], marker='.', c=colorlist, s=markersize, zorder=1);
        ax.legend(handles=patches)
    else:
        ax.scatter(stream[:,0], stream[:,1], marker='.', c='red', s=markersize, zorder=1);
    ax.plot(stream[:,0], stream[:,1], color='gray', linewidth=linewidth, zorder=0);
    if not ax:
        plt.show()
    
    
def plot_signal(signal, stream=None, labels=None, figsize=(11, 2), 
                color='red', linestyle='-', linewidth=0.5, ax=None):
    if not ax:
        fig, ax = plt.subplots(figsize=figsize, dpi=200)
    if labels is not None:
        ax.set_yticks(ticks=range(len(labels)), labels=labels)
    else:
        ax.set_yticks(ticks=np.unique(signal))
    if stream is not None:
        ax.plot(stream[:,0], signal, color=color, linestyle=linestyle, linewidth=linewidth);
    else:
        ax.plot(signal, color=color, linestyle=linestyle, linewidth=linewidth);
    if not ax:
        plt.show()
    
# implement options
def plot_comparison(stream, true_signal, predicted_signal, labels=None, colors=None, 
                    figsize=(11, 11), markersize=1, linewidth=0.1):
    fig, axs = plt.subplots(nrows=3, ncols=1, figsize=figsize, dpi=200,
                           gridspec_kw={'height_ratios':[7,7,3]})
    plot_stream(stream, signal=true_signal, labels=labels, colors=colors, ax=axs[0])
    plot_stream(stream, signal=predicted_signal, labels=labels, colors=colors, ax=axs[1])
    plot_signal(true_signal, stream=stream, color='blue', labels=labels, ax=axs[2])
    plot_signal(predicted_signal, stream=stream, color='red', linestyle=':', labels=labels, ax=axs[2])